In [1]:
import httplib

from urllib import urlencode

headers = {
    "Accept": "*/*",
    "REFERER": "http://xxfb.hydroinfo.gov.cn/svg/svghtml.html",
    "Accept-Encoding": "gzip, deflate",
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E)",
    "Host": "xxfb.hydroinfo.gov.cn",
    #"Connection": "Keep-Alive",
    #"Cookie": "JSESSIONID=3DF01E7575C0A20AC82A2483F9A0FE90.tomcat1",
}

params = {
    "gcxClass": "1",
    "gcxKind": "2",
    "DateL": "2016-01-15",
    "DateM": "2016-01-19",
    "gcxData": "7",
    "site": "11203200",
}

host = "xxfb.hydroinfo.gov.cn"

url = "/svg/svgwait.jsp" + "?" + urlencode(params)

print url

conn = httplib.HTTPConnection(host)
#conn.request("GET", url, headers = headers)
#res = conn.getresponse()
#data = res.read()

#print res.status
#print data

conn.close()

/svg/svgwait.jsp?gcxClass=1&site=11203200&gcxData=7&gcxKind=2&DateM=2016-01-19&DateL=2016-01-15


In [2]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

sites_list = {
    # river basins, province, river, site name, site id
    u"10701210": [u"松辽", u"黑龙江",u"松花江", u"哈尔滨", u"10701210"],
    u"10901900": [u"松辽", u"吉林", u"第二松花江", u"扶余", u"10901900"],
}

def load_sites_list():
    with open('sites.txt') as f:
        for line in f.readlines():
            river_basins, province, river, site_name, site_id = line.split()
            sites_list [site_id] = (river_basins, province, river, site_name)

def get_site_info(site_id):
    return sites_list.get(str(site_id), None)

get_site_info(10701210)

[u'\u677e\u8fbd',
 u'\u9ed1\u9f99\u6c5f',
 u'\u677e\u82b1\u6c5f',
 u'\u54c8\u5c14\u6ee8',
 u'10701210']

In [3]:
import datetime
import copy
import codecs
import xml.etree.ElementTree as ET


HEADERS = {
    "Accept": "*/*",
    "REFERER": "http://xxfb.hydroinfo.gov.cn/svg/svghtml.html",
    "Accept-Encoding": "gzip, deflate",
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E)",
    "Host": "xxfb.hydroinfo.gov.cn",
    #"Connection": "Keep-Alive",
    #"Cookie": "JSESSIONID=3DF01E7575C0A20AC82A2483F9A0FE90.tomcat1",
}

PARAMS = {
    "gcxClass": "1",
    "gcxKind": "2",
    "DateL": "",
    "DateM": "",
    "gcxData": "7",
    "site": "",
}

HOST = "xxfb.hydroinfo.gov.cn"

def build_headers():
    global EHADERS
    return copy.deepcopy(HEADERS)

def build_params(site_id, start_date):
    global PARAMS
    params = copy.deepcopy(PARAMS)
    params["DateL"] = str(start_date)
    params["DateM"] = datetime.date.today()
    params["site"]  = str(site_id)
    return params

def request(method, host, url, params, headers):
    url = "/svg/svgwait.jsp" + "?" + urlencode(params)  
    
    logging.info("request host={host}, url={url}, headers={headers}".format(host=host, url=url, headers=headers))

    url = "/svg/svgwait.jsp" + "?" + urlencode(params)    
    conn = httplib.HTTPConnection(host)
    conn.request("GET", url, headers = headers)
    res = conn.getresponse()
    data = res.read()
    conn.close()
    
    return (res.status, data)
    
def get_monitor_xml_data(site_id, start_date):
    logging.info("get_monitor_xml_data site_id={site_id}, start_date={start_date}".format(
                 site_id=site_id,start_date=start_date))
    
    headers = build_headers()
    params = build_params(site_id, start_date)
    
    status, data = request("GET", HOST, "/svg/svgwait.jsp", params, headers)
    
    if status != 200:
        raise Exception("request_error status=%s" % status)
    
    return codecs.decode("\n".join(data.splitlines()[4:-1]).strip(), "gb2312")

# tmp_xml = get_monitor_xml_data(10701210, datetime.date.today() - datetime.timedelta(days=5))

In [4]:
import codecs
import re

re_safety_level  = re.compile(ur"保证水位(?P<safety_level>[\d\.]+)")
re_warning_level = re.compile(ur"警戒水位(?P<warning_level>[\d\.]+)")
re_water_level_points = re.compile(ur"水位.*points=\"(?P<points>[\d\.,\s]+)")
re_water_flow_points  = re.compile(ur"流量.*points=\"(?P<points>[\d\.,\s]+)")


def parse_points(raw_str, start_date):
    ret = []
    start_date = datetime.datetime(*(start_date.timetuple()[:6])) # convert to datetime

    for p in raw_str.split():
        hours_delta, value = p.split(",")
        time = start_date + datetime.timedelta(hours=float(hours_delta))
        value = float(value)
        ret.append((time, value))
    return ret

def get_monitor_data(site_id, start_date=datetime.date.today() - datetime.timedelta(days=5)):
    logging.info("get_mnitor_data site_id={site_id} start_date={start_date}".format(site_id=site_id, start_date=start_date))
    xml = get_monitor_xml_data(site_id, start_date)
    #xml = tmp_xml

    safety_level = None
    warning_level = None
    water_level = []
    water_flow = []
    
    sl_m = re_safety_level.search(xml)
    if sl_m:
        safety_level = float(sl_m.group("safety_level"))
    else:
        logging.info("not_got_safety_level site_id=%s" % site_id)
    
    wl_m = re_warning_level.search(xml)
    if wl_m:
        warning_level = float(wl_m.group("warning_level"))
    else:
        logging.info("not_got_warning_level site_id=%s" % site_id)
        
    wlp_m = re_water_level_points.search(xml)
    if wlp_m:
        water_level = parse_points(wlp_m.group("points"), start_date)
    else:
        logging.info("not_got_water_level site_id=%s" % site_id)
        
    wfp_m = re_water_flow_points.search(xml)
    if wfp_m:
        water_flow = parse_points(wfp_m.group("points"), start_date)
    else:
        logging.info("not_got_water_flow site_id=%s" % site_id)
        
    return (safety_level, warning_level, water_level, water_flow)
    print safety_level
    print warning_level
    print water_level
    print water_flow

# get_monitor_data("10801400", datetime.date(2010, 1, 20))

In [7]:
import sqlite3
conn = sqlite3.connect("water_regime.db")

def insert_points(table, col, sid, points, cur):
    sql = "INSERT INTO %s(sid, date, %s) VALUES(?, ?, ?)" % (table, col)
    logging.info("execute_sql_insert_points sql=%s table=%s col=%s" % (sql, table, col))
    
    points = map(lambda x: (sid,) + x, points)
    cur.executemany(sql, points)

def initial_import():
    cur = conn.cursor()
    cur.execute("SELECT id, site_id FROM site")

    for line in cur.fetchmany(1):
        rid, site_id = line[0], line[1]
        safety_level, warning_level, water_level, water_flow = get_monitor_data(site_id, datetime.date(2010, 1, 20))
        
        logging.info("import_initial_data site_id={site_id}".format(site_id=site_id))
          
        if safety_level:
            cur.execute("UPDATE site SET safety_level=? WHERE id=?", (safety_level, rid))
        if warning_level:
            cur.execute("UPDATE site SET warning_level=? WHERE id=?", (warning_level, rid))
        if water_level:
            insert_points("water_level", "level", site_id, water_level, cur)
        if water_flow:
            insert_points("water_flow", "flow", site_id, water_flow, cur)
        
        conn.commit()
        
initial_import()

In [6]:
conn.close()